Ora faremo un text classification con il bert.
Importeremo il dataset imdb e poi chiameremo l'oggetto dataset di nlp di hugginface che dividera per noi
i dati in training set e test set.

In [1]:
!pip install nlp==0.4.0
!pip install transformer==3.5.1
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

ERROR: Could not find a version that satisfies the requirement transformer==3.5.1
ERROR: No matching distribution found for transformer==3.5.1
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from nlp import load_dataset
import torch
import numpy as np
import pandas as pd

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#!wget https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc- -O mario.csv

In [5]:
dataset = load_dataset('csv', data_files='../data/raw/imdb.csv', split='train')


Using custom data configuration default


In [6]:
type(dataset)

nlp.arrow_dataset.Dataset

In [7]:
dataset = dataset.train_test_split(test_size=0.3)


In [8]:
dataset

{'train': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 70),
 'test': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 30)}

In [9]:
train_set = dataset['train']
test_set = dataset['test']

Ora importiamo bert, per il task di classificazione dobbiamo scaricare una versione aposita del bert che si chiama
bertForSequenceClassification e deve essere pre allenato e scegliamo la versione base di bert con bert-base-uncased.


In [10]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Va in portato anche un tokenizer, nella pratica non si usa bertTokenizer ma BertTokenizerFast che in colpo solo
tokeniza e da la maschera di attenzione e i tokens id.

In [11]:
from transformers import BertTokenizerFast

In [12]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [13]:
tokenizer('I love Paris')

{'input_ids': [101, 1045, 2293, 3000, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

Con questo tokenizer possiamo anche tokenizare insiemi di frasi, e possiamo anche chiederli di usare
padding inserendo il token [PAD] settando il flag a True chiamtto padding e impostando un valore sulla sequenza
massima.
Dal momento che sono singole frasi e non hanno frasi conseguenti il 'token_type_ids per ciascune di esse è
sempre 0 per ogni token che compone la frase.

In [14]:
tokenizer(['I love Paris', 'birds fly', 'snow fall' ], padding= True, max_length= 5)

{'input_ids': [[101, 1045, 2293, 3000, 102], [101, 5055, 4875, 102, 0], [101, 4586, 2991, 102, 0]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 1, 0]]}

In [15]:
def preProcessing(data):
    return tokenizer(data['text'], padding=True, truncation= True)

ora applichiamo preprocessing ad ogni elemento del training e test set la nostra funzione di preprocessing
attraverso il mapping.

train_set = train_set.map(preprocess, batched=True,
                          batch_size=len(train_set))
test_set = test_set.map(preprocess, batched=True, batch_size=len(test_set))

train_set.set_format('torch',
                      columns=['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch',
                     columns=['input_ids', 'attention_mask', 'label'])


In [16]:
train_set = train_set.map(preProcessing, batched=True, batch_size=len(train_set))
test_set = train_set.map(preProcessing, batched=True, batch_size=len(train_set))

selezioniamo i dati che ci servono con set_format prendendp le colonne necessarie ovvero:
-input_ids
-attention_mask
-label
infine passiamo come argomento anche 'torch' per richiedere i dati in tensori ma per pytorch

In [17]:
train_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [18]:
train_set

Dataset(features: {'label': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 70)

## Formazione del modello
Adesso dobbiamo allenare il nostro bert al compito di classificazione per sentiment analysis che ricordiamo
è considerato un task downstream.
Per prima cosa dobbiamo importare gli oggetti Trainer, TrainingArguments dalla libreria transformers.

In [19]:
from transformers import Trainer, TrainingArguments

Impostiamo la dimensione di batch e il numero di epoche, il warm_up e la regolarizzazione L2 weight_decay

In [20]:
batch_size = 8
epochs = 2
warm_up_steps = 500
weight_decay = 0.01

In [21]:
training_args = TrainingArguments(per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size= batch_size,
                                  num_train_epochs = epochs,
                                  warmup_steps= warm_up_steps,
                                  weight_decay=weight_decay,
                                  output_dir="../output",
                                  logging_dir="../logs")


In [22]:
trainer = Trainer(model = model,
                  args= training_args,
                  train_dataset= train_set,
                  eval_dataset = test_set)


In [23]:
trainer.train()

/home/matteo/PycharmProjects/Longformer/venv/lib/python3.8/site-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)


RuntimeError: CUDA out of memory. Tried to allocate 18.00 MiB (GPU 0; 7.80 GiB total capacity; 6.41 GiB already allocated; 25.19 MiB free; 6.45 GiB reserved in total by PyTorch)